In [1]:
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree   import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
# Random forest
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
# compare standalone models for binary classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from matplotlib import pyplot
from sklearn.ensemble import StackingClassifier
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv1D, LSTM, Dense, Dropout, TimeDistributed, Flatten
from keras.optimizers import Adam
import keras
# Cross-validation
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [51]:
data_path = "./data"
X = pd.read_csv(f'{data_path}/X_cleaned.csv',sep='|')
y = pd.read_csv(f'{data_path}/y_cleaned.csv',sep='|')
x_testing = pd.read_csv(f'{data_path}/X_testing.csv',sep='|')

In [54]:
print(X.shape)
print(y.shape)

(406708, 12)
(406708, 1)


In [83]:
#X = X.drop(columns=["id"],inplace=True)
#y = y.drop(columns=["id"],inplace=True)

In [55]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 10, stratify=y['Cover_Type'])



In [56]:
# For creating the models
print("x_train: ",x_train.shape)
print("y_train: ",y_train.shape)
print("x_test: ",x_test.shape)
print("y_test: ",y_test.shape)
print("x_testing: ",x_testing.shape)

x_train:  (284695, 12)
y_train:  (284695, 1)
x_test:  (122013, 12)
y_test:  (122013, 1)
x_testing:  (406708, 12)


In [57]:
X_train = copy.deepcopy(x_train)
Y_train = copy.deepcopy(y_train)
X_test = copy.deepcopy(x_test)
Y_test = copy.deepcopy(y_test)
X_testing = copy.deepcopy(x_testing)

In [58]:
# After performing copy with dtype = int64
print("x_train: ",X_train.shape)
print("y_train: ",Y_train.shape)
print("x_test: ",X_test.shape)
print("y_test: ",Y_test.shape)
print("x_testing: ",X_testing.shape)

x_train:  (284695, 12)
y_train:  (284695, 1)
x_test:  (122013, 12)
y_test:  (122013, 1)
x_testing:  (406708, 12)


#### Models

In [59]:
def train_using_knn(X_train, y_train,i):
    knn_model = KNeighborsClassifier(n_neighbors=i,leaf_size=1,p=1)
    knn_model.fit(X_train, y_train)
    return knn_model

In [60]:
def prediction(X_test, clf_object):
    y_pred = clf_object.predict(X_test)
    print("Predicted values: ")
    print(y_pred)
    return y_pred
def calculate_Acc(y_test, y_pred):
    print("Confusion matrix: \n")
    confusion_matrix(y_test, y_pred)
    print("Accuracy: ", accuracy_score(y_test,y_pred)*100)
    print("Report ", classification_report(y_test, y_pred))

In [61]:
KNN_model = train_using_knn(X_train, Y_train,3)
#clf_gini = train_using_gini(X_train, Y_train)
#clf_entropy = train_using_entropy(X_train, Y_train)

###  1. Models using all features as default:
Without any type of analysis or feature selection.

KNN

In [62]:
# Using splited dataset
print("X_test: ", X_test.shape)
print("Y_test: ", Y_test.shape)
y_knn_real = prediction(X_test, KNN_model)
print(accuracy_score(Y_test, y_knn_real))

X_test:  (122013, 12)
Y_test:  (122013, 1)
Predicted values: 
[1 7 3 ... 2 2 2]
0.9625777581077426


#Testing with different number of neighborrs
for i in range(20):
    KNN_model = train_using_knn(X_train, Y_train,i+1)
    y_pred = KNN_model.predict(X_test)
    print("neighbors: ",i+1," acc: ",accuracy_score(Y_test, y_pred))

From the above section, we obtained the best results when neighbors=3

In [64]:
# Show params
print(classification_report(Y_test, y_pred))
#roc_auc_score(Y_test,y_pred)

              precision    recall  f1-score   support

           1       0.93      0.94      0.94     44487
           2       0.94      0.95      0.95     59493
           3       0.91      0.94      0.92      7508
           4       0.91      0.67      0.77       577
           5       0.89      0.72      0.80      1994
           6       0.89      0.82      0.85      3647
           7       0.97      0.90      0.93      4307

    accuracy                           0.93    122013
   macro avg       0.92      0.85      0.88    122013
weighted avg       0.93      0.93      0.93    122013



In [66]:
y_pred = KNN_model.predict(x_testing)
y_pred.shape

(406708,)

Store to Kaggle CSV format

In [67]:
import numpy
column_names ={'id': range(len(y_pred)),
 'Cover_type':y_pred}

labels = pd.DataFrame(column_names)

labels.to_csv("Sebastian_Cajas.csv", sep=",",index=False)
print(labels)

            id  Cover_type
0            0           1
1            1           2
2            2           2
3            3           1
4            4           1
...        ...         ...
406703  406703           2
406704  406704           6
406705  406705           2
406706  406706           2
406707  406707           7

[406708 rows x 2 columns]


#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]
#Convert to dictionary
hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)
#Create new KNN object
knn_2 = KNeighborsClassifier()
#Use GridSearch
clf = GridSearchCV(knn_2, hyperparameters, cv=10)
#Fit the model
best_model = clf.fit(X_train,Y_train)

### Cross-val K-fold

In [74]:
from sklearn import model_selection
from sklearn.model_selection import KFold,cross_val_score
kfold = model_selection.KFold(n_splits=10, random_state=100,shuffle=True)
kfold

KFold(n_splits=10, random_state=100, shuffle=True)

In [41]:
results_kfold = model_selection.cross_val_score(KNN_model, X_test, Y_test, cv=kfold)
print("Accuracy: %.2f%%" % (results_kfold.mean()*100.0))

Accuracy: 93.52%


### Feature selection N1

In [69]:
fields = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']
X_change = X_train[fields]
x_test_change= X_test[fields]

print(X_change.shape)
print(Y_train.shape)
print(x_test_change.shape)
print(Y_test.shape)

KNN_model = train_using_knn(X_change, Y_train,3)
y_knn_real = prediction(x_test_change, KNN_model)
print(accuracy_score(Y_test, y_knn_real))


(284695, 10)
(284695, 1)
(122013, 10)
(122013, 1)
Predicted values: 
[1 7 3 ... 2 2 2]
0.9608074549433258


In [70]:
print("Modelo final tiene dimensiones: ",y_knn_real.shape)

Modelo final tiene dimensiones:  (122013,)


In [71]:
import xgboost as xgb

In [72]:
import xgboost as xgb
from xgboost import XGBClassifier
print('> Training XGBoost classifier...')
params={
            'reg_alpha': 23, 
            'max_depth': 8, 
            'learning_rate': 0.01, 
            'gamma': 1.0
        }
data_dmatrix = xgb.DMatrix(data=X_train,label=Y_train)
xg_reg = XGBClassifier(random_state=0, booster='gbtree', **params)


> Training XGBoost classifier...


In [73]:
xg_reg.fit(X_train,Y_train)
preds = xg_reg.predict(X_test)
print(preds.shape)
print(preds)


(122013,)
[1 7 3 ... 2 2 2]


In [75]:
print(accuracy_score(Y_test, preds))

0.7840066222451706
